<a href="https://colab.research.google.com/github/Sameera326/GenerativeAIB40/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Load dataset
df = pd.read_excel('Data_Solar Radiation.xlsx', header=1)

# Rename last column to Solar_Radiation
df.columns = df.columns[:-1].tolist() + ['Solar_Radiation']

# Convert numeric columns
for col in df.columns:
    if col not in ['From Date', 'To Date']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop missing values
df = df.dropna()

# Features and target
X = df.drop(columns=['From Date', 'To Date', 'Solar_Radiation'])
y = df['Solar_Radiation']

# Normalize
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

# ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=200, batch_size=16,
          validation_split=0.1, callbacks=[early_stop], verbose=1)

# Predict and evaluate
y_pred = model.predict(X_test).flatten()
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
accuracy_within_10 = np.mean(np.abs(y_test - y_pred) <= 10) * 100

print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.4f}")
print(f"Accuracy (±10 units): {accuracy_within_10:.2f}%")

# Save as TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("solar_radiation_model.tflite", "wb") as f:
    f.write(tflite_model)


Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 7986.9189 - mae: 86.5650 - val_loss: 10716.3105 - val_mae: 97.2004
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8127.9248 - mae: 87.3345 - val_loss: 10150.8096 - val_mae: 94.2840
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7704.1685 - mae: 84.7520 - val_loss: 8560.2061 - val_mae: 85.5647
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5488.0771 - mae: 70.4824 - val_loss: 5181.6929 - val_mae: 64.3618
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2705.8740 - mae: 46.2163 - val_loss: 1674.2742 - val_mae: 31.0219
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 745.1612 - mae: 20.6099 - val_loss: 1342.8779 - val_mae: 26.7443
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 779.3398 - mae: 20.8893 - val_loss: 1437.8234 - val_mae: 28.2940
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 778.8994 - mae: 20.9786 - val_loss: 1341.3181 - val_mae: 26.6722
Epoch 9/200


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Load dataset
df = pd.read_excel('Data_Solar Radiation.xlsx', header=1)
df.columns = df.columns[:-1].tolist() + ['Solar_Radiation']

# Clean and convert
for col in df.columns:
    if col not in ['From Date', 'To Date']:
        # Attempt to convert to numeric, errors to NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
        # If conversion fails, try to convert to datetime
        # But only if the column is still an object type after the first conversion:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_datetime(df[col])
                # Extract relevant features from datetime
                df[col + '_hour'] = df[col].dt.hour
                df[col + '_day'] = df[col].dt.day
                df[col + '_month'] = df[col].dt.month
                df = df.drop(columns=[col])  # Drop original datetime column
            except ValueError:
                pass  # If datetime conversion fails, skip

# Drop rows with NaNs *after* all conversions
df.dropna(inplace=True)

# Explicitly select only numeric features for correlation
numeric_features = df.select_dtypes(include=np.number).columns.tolist()
# Remove 'Solar_Radiation' as it's the target
numeric_features.remove('Solar_Radiation')

# Now you can calculate correlations on numeric features only
correlation = df[numeric_features].corrwith(df['Solar_Radiation']).abs().sort_values(ascending=False)
top_features = correlation[0:17].index.tolist()  # top 17 features


# ... (Rest of your code) ...

# Feature matrix and target
X = df[top_features]
y = df['Solar_Radiation']

# Standardize (works better with ReLU)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

# Deep ANN Model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model.fit(X_train, y_train, epochs=1000, batch_size=8,
          validation_split=0.15, callbacks=[early_stop], verbose=1)

# Evaluate
y_pred = model.predict(X_test).flatten()
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
custom_accuracy = np.mean(np.abs(y_test - y_pred) / y_test <= 0.10) * 100

# Display Results
print(f"\n✅ Final Model Evaluation:")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.4f}")
print(f"Custom Accuracy (±10%): {custom_accuracy:.2f}%")

# Save as TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("solar_radiation_high_accuracy.tflite", "wb") as f:
    f.write(tflite_model)

print("\n💾 Model saved as 'solar_radiation_high_accuracy.tflite'")

Epoch 1/1000


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 6728.2612 - mae: 76.7560 - val_loss: 1824.7640 - val_mae: 29.6444
Epoch 2/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1352.7152 - mae: 25.4205 - val_loss: 1270.9331 - val_mae: 29.2291
Epoch 3/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 573.8896 - mae: 19.1825 - val_loss: 940.3633 - val_mae: 22.1245
Epoch 4/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 394.9984 - mae: 15.2745 - val_loss: 863.0643 - val_mae: 19.7692
Epoch 5/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 449.6913 - mae: 16.3733 - val_loss: 875.9691 - val_mae: 21.8664
Epoch 6/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 402.2251 - mae: 15.5814 - val_loss: 810.5679 - val_mae: 18.9064
Epoch 7/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 370.5264 - mae: 15.7356 - val_loss: 794.8168 - val_mae: 18.8838
Epoch 8/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 346.4727 - mae: 14.4537 - val_loss: 773.4407 - val_mae: 18.0917
Epoch 9/1000
38/